In [1]:
#importing dependencies
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to C:\Users\Shubham
[nltk_data]     Hajare\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Load data
file = open('FileInput.txt').read()

In [3]:
#Tokenization
#Standardization
def tokenize_words(input):
    input = input.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return "".join(filtered)
processed_inputs = tokenize_words(file)

In [4]:
#Chars to numbers
chars = sorted(list(set(processed_inputs)))
chars_to_num = dict((c, i) for i, c in enumerate(chars))

In [5]:
#Check if words to chars or chars to num has worked?
inp_len = len(processed_inputs)
vocab_len = len(chars)
print("Total number of characters = ",inp_len)
print("Total vocab= ",vocab_len)

Total number of characters =  140578
Total vocab=  41


In [6]:
#Sequence length
seq_len = 100
x_data = []
y_data = []
#Loop through the sequence
for i in range(0, inp_len-seq_len, 1):
    in_seq = processed_inputs[i: i + seq_len]
    out_seq = processed_inputs[i + seq_len]
    x_data.append([chars_to_num[char] for char in in_seq])
    y_data.append(chars_to_num[out_seq])
n_patterns = len(x_data)
print("Total Patterns = ",n_patterns)

Total Patterns =  140478


In [7]:
#Convert input sequence to np array
X = numpy.reshape(x_data, (n_patterns, seq_len, 1))
X = X/float(vocab_len)

In [8]:
#One hot encoding our label data
y = np_utils.to_categorical(y_data)

In [11]:
#Creating a sequential model
model = Sequential()
model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [12]:
#Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
#Saving weights
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [14]:
#Fit the model and let it train
model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
549/549 [==============================] - ETA: 0s - loss: 3.0200
Epoch 00001: loss improved from inf to 3.02003, saving model to model_weights_saved.hdf5
549/549 [==============================] - 2425s 4s/step - loss: 3.0200
Epoch 2/4
549/549 [==============================] - ETA: 0s - loss: 2.9951
Epoch 00002: loss improved from 3.02003 to 2.99506, saving model to model_weights_saved.hdf5
549/549 [==============================] - 1567s 3s/step - loss: 2.9951
Epoch 3/4
549/549 [==============================] - ETA: 0s - loss: 2.9904
Epoch 00003: loss improved from 2.99506 to 2.99043, saving model to model_weights_saved.hdf5
549/549 [==============================] - 3022s 6s/step - loss: 2.9904
Epoch 4/4
549/549 [==============================] - ETA: 0s - loss: 2.9791
Epoch 00004: loss improved from 2.99043 to 2.97908, saving model to model_weights_saved.hdf5
549/549 [==============================] - 4058s 7s/step - loss: 2.9791


In [15]:
#Recompile model with saved weights
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
#Output of model back to characters
num_to_char = dict((i,c) for i,c in enumerate(chars))
#Random seed to help generate
start = numpy.random.randint(0, len(x_data)-1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" nsenessscrutinygrownunevenjerksbalanceoctober1stalmostfourhundredthousanddollarstradesbobconsultedot "


In [17]:
#Generate the text
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x/float(vocab_len)
    pred = model.predict(x, verbose=0)
    index = numpy.argmax(pred)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee